In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('allTogether.csv')

# remove ids and numbers
df = df.drop(['_id','question.number','user._id'], axis=1)
# groupNames, majLabel and colors are synonyms
df = df.drop(['question.groupNames','question.colors'], axis=1)

# add absolute version of the cv score (demographic parity)
df['question.abs_cv'] = df['question.cv']
df.loc[df['question.cv']>1, 'question.abs_cv'] = 2-df['question.cv']
# add absolute versions of comparative accuracy scores
df['question.abs_s_tnr'] = df['question.s_tnr']
df.loc[df['question.s_tnr']>1, 'question.abs_s_tnr'] = 2-df['question.s_tnr']
df['question.abs_s_tpr'] = df['question.s_tpr']
df.loc[df['question.s_tpr']>1, 'question.abs_s_tpr'] = 2-df['question.s_tpr']
df['question.abs_s_bcr'] = df['question.s_bcr']
df.loc[df['question.s_bcr']>1, 'question.abs_s_bcr'] = 2-df['question.s_bcr']
# add absolute versions of comparative adjusted accuracy scores
df['question.abs_s_adj_tnr'] = df['question.s_adj_tnr']
df.loc[df['question.s_adj_tnr']>1, 'question.abs_s_adj_tnr'] = 2-df['question.s_adj_tnr']
df['question.abs_s_adj_tpr'] = df['question.s_adj_tpr']
df.loc[df['question.s_adj_tpr']>1, 'question.abs_s_adj_tpr'] = 2-df['question.s_adj_tpr']
df['question.abs_s_adj_bcr'] = df['question.s_adj_bcr']
df.loc[df['question.s_adj_bcr']>1, 'question.abs_s_adj_bcr'] = 2-df['question.s_adj_bcr']
pd.set_option('display.max_columns', None)
df.head()

,rating,inconfidence,secondsElapsed,question.questionType,question.minSize,question.majSize,question.majLabel,question.tpr,question.tnr,question.bcr,question.accuracy,question.adj_tpr,question.adj_tnr,question.adj_bcr,question.adj_accuracy,question.tpr0,question.tnr0,question.bcr0,question.accuracy0,question.adj_tpr0,question.adj_tnr0,question.adj_bcr0,question.adj_accuracy0,question.tpr1,question.tnr1,question.bcr1,question.accuracy1,question.adj_tpr1,question.adj_tnr1,question.adj_bcr1,question.adj_accuracy1,question.s_tpr,question.s_tnr,question.s_bcr,question.s_accuracy,question.s_adj_tpr,question.s_adj_tnr,question.s_adj_bcr,question.s_adj_accuracy,question.cv,question.cluster,question.ordering_utility,question.sum_of_diff,question.rND,question.new_rND,user.language,user.accuracy,user.age,user.believe,user.comparativeAccuracy,user.confidence,user.edu,user.fear,user.gender,user.genderParity,user.political,user.religious,user.screenHeight,user.screenWidth,user.will,user.agreeableness,user.conscientiousness,user.extraversion,user.neuroticism,user.openness,question.abs_cv,question.abs_s_tnr,question.abs_s_tpr,question.abs_s_bcr,question.abs_s_adj_tnr,question.abs_s_adj_tpr,question.abs_s_adj_bcr
0,0.89,0.84,36.534,selection,3,7,female,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,NaN,1.00,1.000000,NaN,NaN,1.000000,1.0,1.0,1.00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,NaN,1.000000,1.000000,NaN,NaN,1.000000,1.714286,3,NaN,NaN,NaN,NaN,de,0.86,25,0.37,0.34,0.74,abitur,0.5,male,0.73,0.78,0.13,1080,1920,0.82,3.0,1.5,3.5,2.5,4.0,0.285714,NaN,1.000000,NaN,NaN,1.000000,NaN
1,0.58,NaN,16.849,selection,5,5,female,0.833333,0.750000,0.791667,0.8,0.927083,0.833333,0.880208,0.898551,0.666667,1.000000,0.833333,0.80,0.854167,1.000000,0.927083,0.898551,1.0,0.5,0.75,0.800000,1.000000,0.666667,0.833333,0.898551,0.666667,1.500000,1.083333,1.000000,0.854167,1.333333,1.093750,1.000000,0.600000,8,NaN,NaN,NaN,NaN,de,0.86,25,0.37,0.34,0.74,abitur,0.5,male,0.73,0.78,0.13,1080,1920,0.82,3.0,1.5,3.5,2.5,4.0,0.600000,0.500000,0.666667,0.916667,0.666667,0.854167,0.906250
2,0.39,0.06,12.849,selection,4,6,male,0.666667,0.857143,0.761905,0.8,0.774194,0.885246,0.829720,0.862745,1.000000,0.666667,0.833333,0.75,1.000000,0.791045,0.895522,0.827160,0.5,1.0,0.75,0.833333,0.730769,1.000000,0.865385,0.921348,1.500000,0.666667,1.083333,0.916667,1.269231,0.791045,1.030138,0.905812,1.333333,7,NaN,NaN,NaN,NaN,de,0.86,25,0.37,0.34,0.74,abitur,0.5,male,0.73,0.78,0.13,1080,1920,0.82,3.0,1.5,3.5,2.5,4.0,0.666667,0.666667,0.500000,0.916667,0.791045,0.730769,0.969862
3,0.25,NaN,10.424,selection,4,6,female,0.500000,0.666667,0.583333,0.6,0.565217,0.813725,0.689471,0.736486,0.333333,1.000000,0.666667,0.50,0.395349,1.000000,0.697674,0.500000,1.0,0.6,0.80,0.666667,1.000000,0.747967,0.873984,0.770370,0.333333,1.400000,0.866667,0.833333,0.395349,1.252033,0.823691,0.729630,0.750000,2,NaN,NaN,NaN,NaN,de,0.86,25,0.37,0.34,0.74,abitur,0.5,male,0.73,0.78,0.13,1080,1920,0.82,3.0,1.5,3.5,2.5,4.0,0.750000,0.600000,0.333333,0.866667,0.747967,0.395349,0.823691
4,0.05,0.13,8.199,selection,1,9,female,0.000000,0.888889,0.444444,0.8,0.000000,0.857143,0.428571,0.825911,NaN,0.000000,NaN,0.00,NaN,0.000000,NaN,0.000000,0.0,1.0,0.50,0.888889,0.000000,1.000000,0.500000,0.957746,NaN,0.000000,NaN,0.111111,NaN,0.000000,NaN,0.042254,2.000000,0,NaN,NaN,NaN,NaN,de,0.86,25,0.37,0.34,0.74,abitur,0.5,male,0.73,0.78,0.13,1080,1920,0.82,3.0,1.5,3.5,2.5,4.0,0.000000,0.000000,NaN,NaN,0.000000,NaN,NaN


In [3]:
cat_cols = [
    'question.majLabel',
    'question.questionType',
    'user.language',
    'user.age',
    'user.edu',
    'user.gender',
]
num_cols = [
    'secondsElapsed',
    'question.minSize',
    'question.majSize',
    'user.believe',
    'user.confidence',
    'user.fear',
    'user.political',
    'user.religious',
    'user.screenHeight',
    'user.screenWidth',
    'user.will',
    'user.agreeableness',
    'user.conscientiousness',
    'user.extraversion',
    'user.neuroticism',
    'user.openness',
]
num_cols_sel = [
    'question.tpr',
    'question.tnr',
    'question.bcr',
    'question.accuracy',
    'question.adj_tpr',
    'question.adj_tnr',
    'question.adj_bcr',
    'question.adj_accuracy',
    'question.tpr0',
    'question.tnr0',
    'question.bcr0',
    'question.accuracy0',
    'question.adj_tpr0',
    'question.adj_tnr0',
    'question.adj_bcr0',
    'question.adj_accuracy0',
    'question.tpr1',
    'question.tnr1',
    'question.bcr1',
    'question.accuracy1',
    'question.adj_tpr1',
    'question.adj_tnr1',
    'question.adj_bcr1',
    'question.adj_accuracy1',
    'question.s_tpr',
    'question.s_tnr',
    'question.s_bcr',
    'question.s_accuracy',
    'question.s_adj_tpr',
    'question.s_adj_tnr',
    'question.s_adj_bcr',
    'question.s_adj_accuracy',
    'question.abs_s_tpr',
    'question.abs_s_tnr',
    'question.abs_s_bcr',
    'question.abs_s_adj_tpr',
    'question.abs_s_adj_tnr',
    'question.abs_s_adj_bcr',
    'question.cv',
    'question.abs_cv',
]
num_cols_ran = [
    'question.ordering_utility',
    'question.sum_of_diff',
    'question.rND',
]
# don't use the following attributes in the model
    #'inconfidence',
    #'question.cluster',
    #'user.accuracy',
    #'user.comparativeAccuracy',
    #'user.genderParity',

# split dataset according to questionType
df_ran = df.query("`question.questionType` == 'ranking'")[['rating'] + num_cols + num_cols_ran + cat_cols]
df_sel = df.query("`question.questionType` == 'selection'")[['rating'] + num_cols + num_cols_sel + cat_cols]
df_sel

,rating,secondsElapsed,question.minSize,question.majSize,user.believe,user.confidence,user.fear,user.political,user.religious,user.screenHeight,user.screenWidth,user.will,user.agreeableness,user.conscientiousness,user.extraversion,user.neuroticism,user.openness,question.tpr,question.tnr,question.bcr,question.accuracy,question.adj_tpr,question.adj_tnr,question.adj_bcr,question.adj_accuracy,question.tpr0,question.tnr0,question.bcr0,question.accuracy0,question.adj_tpr0,question.adj_tnr0,question.adj_bcr0,question.adj_accuracy0,question.tpr1,question.tnr1,question.bcr1,question.accuracy1,question.adj_tpr1,question.adj_tnr1,question.adj_bcr1,question.adj_accuracy1,question.s_tpr,question.s_tnr,question.s_bcr,question.s_accuracy,question.s_adj_tpr,question.s_adj_tnr,question.s_adj_bcr,question.s_adj_accuracy,question.abs_s_tpr,question.abs_s_tnr,question.abs_s_bcr,question.abs_s_adj_tpr,question.abs_s_adj_tnr,question.abs_s_adj_bcr,question.cv,question.abs_cv,question.majLabel,question.questionType,user.language,user.age,user.edu,user.gender
0,0.89,36.534,3,7,0.37,0.74,0.50,0.78,0.13,1080,1920,0.82,3.0,1.5,3.5,2.5,4.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,NaN,1.000000,1.000000,NaN,NaN,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,NaN,1.000000,1.000000,NaN,NaN,1.000000,1.000000,NaN,NaN,1.000000,NaN,NaN,1.714286,0.285714,female,selection,de,25,abitur,male
1,0.58,16.849,5,5,0.37,0.74,0.50,0.78,0.13,1080,1920,0.82,3.0,1.5,3.5,2.5,4.0,0.833333,0.750000,0.791667,0.8,0.927083,0.833333,0.880208,0.898551,0.666667,1.000000,0.833333,0.800000,0.854167,1.000000,0.927083,0.898551,1.0,0.500000,0.750000,0.800000,1.000000,0.666667,0.833333,0.898551,0.666667,1.500000,1.083333,1.000000,0.854167,1.333333,1.093750,1.000000,0.666667,0.500000,0.916667,0.854167,0.666667,0.906250,0.600000,0.600000,female,selection,de,25,abitur,male
2,0.39,12.849,4,6,0.37,0.74,0.50,0.78,0.13,1080,1920,0.82,3.0,1.5,3.5,2.5,4.0,0.666667,0.857143,0.761905,0.8,0.774194,0.885246,0.829720,0.862745,1.000000,0.666667,0.833333,0.750000,1.000000,0.791045,0.895522,0.827160,0.5,1.000000,0.750000,0.833333,0.730769,1.000000,0.865385,0.921348,1.500000,0.666667,1.083333,0.916667,1.269231,0.791045,1.030138,0.905812,0.500000,0.666667,0.916667,0.730769,0.791045,0.969862,1.333333,0.666667,male,selection,de,25,abitur,male
3,0.25,10.424,4,6,0.37,0.74,0.50,0.78,0.13,1080,1920,0.82,3.0,1.5,3.5,2.5,4.0,0.500000,0.666667,0.583333,0.6,0.565217,0.813725,0.689471,0.736486,0.333333,1.000000,0.666667,0.500000,0.395349,1.000000,0.697674,0.500000,1.0,0.600000,0.800000,0.666667,1.000000,0.747967,0.873984,0.770370,0.333333,1.400000,0.866667,0.833333,0.395349,1.252033,0.823691,0.729630,0.333333,0.600000,0.866667,0.395349,0.747967,0.823691,0.750000,0.750000,female,selection,de,25,abitur,male
4,0.05,8.199,1,9,0.37,0.74,0.50,0.78,0.13,1080,1920,0.82,3.0,1.5,3.5,2.5,4.0,0.000000,0.888889,0.444444,0.8,0.000000,0.857143,0.428571,0.825911,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,0.0,1.000000,0.500000,0.888889,0.000000,1.000000,0.500000,0.957746,NaN,0.000000,NaN,0.111111,NaN,0.000000,NaN,0.042254,NaN,0.000000,NaN,NaN,0.000000,NaN,2.000000,0.000000,female,selection,de,25,abitur,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,0.70,35.312,4,6,0.53,0.81,0.21,0.81,0.13,780,360,0.55,4.0,2.0,4.0,3.0,4.5,0.875000,0.500000,0.687500,0.8,0.965753,0.583333,0.774543,0.936709,1.000000,0.000000,0.500000,0.750000,1.000000,0.000000,0.500000,0.863636,0.8,1.000000,0.900000,0.833333,0.929293,1.000000,0.964646,0.933962,1.200000,0.000000,0.600000,0.916667,1.070707,0.000000,0.535354,0.929674,0.800000,0.000000,0.600000,0.929293,0.000000,0.535354,1.333333,0.666667,male,selection,de,25,abitur,male
2696,0.36,11.247,1,9,0.53,0.81,0.21,0.

In [4]:
# selection model
y = df_sel['rating']
X = df_sel[num_cols + num_cols_sel + cat_cols]

categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
numerical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('poly', PolynomialFeatures(degree=2)),
    ('scaler', StandardScaler())
])

preprocessing = ColumnTransformer(
    [('cat', categorical_pipe, cat_cols),
     ('num', numerical_pipe, num_cols + num_cols_sel)])

rr = Pipeline([
    ('preprocess', preprocessing),
    ('classifier', Ridge(alpha=1000))
])

print("Selection Model - test accuracy: %0.3f" % np.mean(cross_val_score(rr, X, y, cv=8, n_jobs=-1)))

Selection Model - test accuracy: 0.532


| degree | alpha | test accuracy |
| --- | --- | --- |
| 3 | 50,000 | 52.9 |
| 2 | 1,000 | 53.2 |
| 1 | 13 | 51.4 |

**Selection Models**: accuracy comparable to random forests with 6 to 7 bins (uniform binning)

In [5]:
# ranking model
y = df_ran['rating']
X = df_ran[num_cols + num_cols_ran + cat_cols]

categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
numerical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('poly', PolynomialFeatures(degree=2)),
    ('scaler', StandardScaler())
])

preprocessing = ColumnTransformer(
    [('cat', categorical_pipe, cat_cols),
     ('num', numerical_pipe, num_cols + num_cols_ran)])

rr = Pipeline([
    ('preprocess', preprocessing),
    ('classifier', Ridge(alpha=5000))
])

print("Ranking Model - test accuracy: %0.3f" % np.mean(cross_val_score(rr, X, y, cv=8, n_jobs=-1)))

Ranking Model - test accuracy: 0.162


| degree | alpha | test accuracy |
| --- | --- | --- |
| 3 | 50,000 | 16.2 |
| 2 | 5,000 | 16.2 |
| 1 | 500 | 14.1 |

**Ranking Models**: accuracy in general worse than the random forests

In [6]:
# global model
y = df['rating']
X = df[num_cols + num_cols_sel + num_cols_ran + cat_cols]

categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
numerical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('poly', PolynomialFeatures(degree=2)),
    ('scaler', StandardScaler())
])

preprocessing = ColumnTransformer(
    [('cat', categorical_pipe, cat_cols),
     ('num', numerical_pipe, num_cols + num_cols_ran + num_cols_sel)])

rr = Pipeline([
    ('preprocess', preprocessing),
    ('classifier', Ridge(alpha=1000))
])

print("Global Model - test accuracy: %0.3f" % np.mean(cross_val_score(rr, X, y, cv=8, n_jobs=-1)))

Global Model - test accuracy: 0.425


| degree | alpha | test accuracy |
| --- | --- | --- |
| 3 | 20,000 | 42.4 |
| 2 | 1,000 | 42.5 |
| 1 | 200 | 40.9 |

**Global Models**: accuracy comparable to random forests with 6 to 7 bins (uniform binning)